In [1]:
import requests
from bs4 import BeautifulSoup
import os


ModuleNotFoundError: No module named 'cv2'

In [ ]:

folder_name = "hico_data"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)


In [ ]:


base_url = "https://oceandata.sci.gsfc.nasa.gov"
main_url = "https://oceandata.sci.gsfc.nasa.gov/directdataaccess/Level-1B/HICO/2014"
num_subdirs = 5  # Number of top subdirectories to explore

# Send a GET request to the main URL
response = requests.get(main_url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

# Find all the links to the subdirectories
subdir_links = [link["href"] for link in soup.find_all("a") if link["href"].endswith("/")]

# Initialize an empty list to store the download links
download_links = []

# Iterate over the top num_subdirs subdirectories
for subdir_link in subdir_links[:num_subdirs]:
    subdir_url = base_url + subdir_link
    
    # Send a GET request to the subdirectory URL
    subdir_response = requests.get(subdir_url)
    
    # Create a BeautifulSoup object to parse the HTML content of the subdirectory
    subdir_soup = BeautifulSoup(subdir_response.content, "html.parser")
    
    # Find all the links to the .bz2 files in the subdirectory
    file_links = [link["href"] for link in subdir_soup.find_all("a") if link["href"].endswith(".bz2")]
    
    # Construct the full download URLs and append them to the download_links list
    download_links.extend([base_url + "/getfile/" + link for link in file_links])

# Save the download links to a text file
with open(folder_name, "w") as file:
    file.write("\n".join(download_links))

print("Download links saved to" +  folder_name+".txt")
